# 🕷️ 웹 크롤링 완전 정복 가이드

## 📋 학습 목표
이 노트북을 완료하면 다음을 할 수 있게 됩니다:
- 웹 크롤링의 기본 원리와 동작 방식 이해
- `requests` 모듈로 웹페이지 데이터 수집
- `BeautifulSoup`로 HTML 문서 파싱 및 데이터 추출
- CSS 선택자를 활용한 정확한 요소 선택
- 실전 웹사이트에서 구조화된 데이터 추출
- 크롤링 시 발생할 수 있는 문제 해결 방법

## 📚 목차
1. **환경 설정 및 패키지 확인**
2. **웹 크롤링 기본 개념과 원리**
3. **HTTP 통신과 requests 모듈 기초**
4. **HTML 구조 이해하기**
5. **BeautifulSoup 기초 파싱**
6. **CSS 선택자와 고급 파싱 기법**
7. **실전 예제: 복합 웹페이지 파싱**
8. **에러 처리와 예외 상황 대응**
9. **크롤링 윤리와 법적 주의사항**
10. **종합 정리 및 실전 프로젝트 제안**

---


## 1. 🔧 환경 설정 및 패키지 확인

웹 크롤링을 위해 필요한 라이브러리들을 확인하고 설치해보겠습니다.

### 필수 패키지
- **requests**: HTTP 요청을 보내고 응답을 받는 라이브러리
- **beautifulsoup4**: HTML 파싱 및 데이터 추출 라이브러리
- **lxml**: BeautifulSoup의 빠른 파서 (선택사항)


In [1]:
# 🔍 패키지 설치 및 버전 확인

# 필요한 패키지들을 import하여 설치 상태 확인
try:
    import requests
    print(f"✅ requests 버전: {requests.__version__}")
except ImportError:
    print("❌ requests가 설치되지 않았습니다. 'pip install requests'로 설치하세요.")

try:
    from bs4 import BeautifulSoup
    import bs4
    print(f"✅ beautifulsoup4 설치됨")
except ImportError:
    print("❌ beautifulsoup4가 설치되지 않았습니다. 'pip install beautifulsoup4'로 설치하세요.")

try:
    import lxml  # type: ignore
    print(f"✅ lxml 설치됨")
except ImportError:
    print("⚠️  lxml이 설치되지 않았습니다 (선택사항). 설치하면 파싱 속도가 향상됩니다.")

# 추가 유용한 패키지들
try:
    import urllib.parse
    print("✅ urllib.parse 사용 가능 (URL 조작용)")
except ImportError:
    pass

try:
    import time
    print("✅ time 모듈 사용 가능 (요청 간격 조절용)")
except ImportError:
    pass

print("\n🎯 모든 필수 패키지가 준비되었습니다!")


✅ requests 버전: 2.32.4
✅ beautifulsoup4 설치됨
✅ lxml 설치됨
✅ urllib.parse 사용 가능 (URL 조작용)
✅ time 모듈 사용 가능 (요청 간격 조절용)

🎯 모든 필수 패키지가 준비되었습니다!


## 2. 🌐 웹 크롤링 기본 개념과 원리

### 웹 크롤링이란?
웹 크롤링(Web Crawling) 또는 웹 스크래핑(Web Scraping)은 웹사이트에서 필요한 정보를 자동으로 수집하는 기술입니다.

### 웹 크롤링의 동작 원리
```markdown
📱 클라이언트(Python) ----HTTP 요청----> 🖥️ 웹서버
                      <---HTML 응답----
                      
1. 요청(Request): 특정 URL에 HTTP 요청을 보냄
2. 응답(Response): 서버에서 HTML, JSON 등의 데이터로 응답
3. 파싱(Parsing): 받은 데이터에서 필요한 정보만 추출
4. 저장(Storage): 추출한 데이터를 파일이나 DB에 저장
```

### 크롤링 도구의 발전
- **urllib** (초기) → **requests** (현재 표준)
- **정규표현식** → **BeautifulSoup/lxml** (HTML 파싱)
- **정적 페이지** → **Selenium** (동적 페이지/JavaScript)

### 크롤링이 어려운 이유
- 웹사이트마다 다른 구조와 방식
- 지속적인 웹사이트 업데이트
- 봇 차단 기술 (reCAPTCHA, 요청 제한 등)
- 동적 콘텐츠 증가 (JavaScript 렌더링)


## 3. 📡 HTTP 통신과 requests 모듈 기초

### HTTP 상태 코드의 이해
웹 서버는 요청에 대한 응답으로 상태 코드를 보냅니다:

| 코드 | 의미 | 설명 |
|------|------|------|
| **2xx** | **성공** | 요청이 성공적으로 처리됨 |
| 200 | OK | 정상적인 응답 |
| **4xx** | **클라이언트 에러** | 요청에 문제가 있음 |
| 404 | Not Found | 페이지를 찾을 수 없음 |
| 403 | Forbidden | 접근 권한이 없음 |
| **5xx** | **서버 에러** | 서버에서 처리 중 오류 발생 |
| 500 | Internal Server Error | 서버 내부 오류 |

### requests 모듈의 주요 기능
- **간단한 API**: `requests.get(url)` 한 줄로 웹페이지 가져오기
- **자동 인코딩**: 응답 데이터의 인코딩 자동 처리
- **세션 관리**: 쿠키와 세션 자동 처리
- **다양한 HTTP 메소드**: GET, POST, PUT, DELETE 등 지원


In [2]:
# 🚀 첫 번째 웹 크롤링 실습

import requests
import time

def fetch_webpage(url):
    """
    웹페이지를 가져오는 안전한 함수
    
    Args:
        url (str): 가져올 웹페이지의 URL
        
    Returns:
        tuple: (성공여부, 응답객체 또는 에러메시지)
    """
    try:
        print(f"🔄 요청 중: {url}")
        
        # User-Agent 헤더 추가 (일부 사이트는 브라우저인 척 해야 함)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        # 요청 보내기 (타임아웃 10초)
        response = requests.get(url, headers=headers, timeout=10)
        
        print(f"📊 응답 상태 코드: {response.status_code}")
        print(f"📏 응답 크기: {len(response.text):,} 문자")
        print(f"🔤 인코딩: {response.encoding}")
        
        # 상태 코드에 따른 처리
        if response.status_code == 200:
            print("✅ 성공적으로 페이지를 가져왔습니다!")
            return True, response
        elif response.status_code == 404:
            print("❌ 페이지를 찾을 수 없습니다 (404)")
            return False, "페이지 없음"
        elif response.status_code == 403:
            print("🚫 접근이 거부되었습니다 (403)")
            return False, "접근 거부"
        else:
            print(f"⚠️  예상치 못한 상태 코드: {response.status_code}")
            return False, f"상태 코드: {response.status_code}"
            
    except requests.exceptions.Timeout:
        print("⏰ 요청 시간이 초과되었습니다")
        return False, "타임아웃"
    except requests.exceptions.ConnectionError:
        print("🌐 네트워크 연결 오류입니다")
        return False, "연결 오류"
    except Exception as e:
        print(f"💥 알 수 없는 오류: {e}")
        return False, str(e)

# 실습용 웹페이지들
test_urls = [
    "http://www.pythonscraping.com/exercises/exercise1.html",  # 성공 예제
    "https://httpbin.org/status/404",  # 404 에러 예제
    "https://httpbin.org/delay/2"  # 지연 응답 예제
]

for i, url in enumerate(test_urls, 1):
    print(f"\n{'='*60}")
    print(f"🧪 테스트 {i}: {url}")
    print('='*60)
    
    success, result = fetch_webpage(url)
    
    if success:
        # 성공한 경우 HTML 일부 출력 (result는 response 객체)
        html_text = result.text  # type: ignore
        html_preview = html_text[:300] + "..." if len(html_text) > 300 else html_text
        print(f"\n📄 HTML 미리보기:\n{html_preview}")
    else:
        print(f"❌ 실패: {result}")
    
    # 서버에 부담을 주지 않기 위해 잠시 대기
    time.sleep(1)



🧪 테스트 1: http://www.pythonscraping.com/exercises/exercise1.html
🔄 요청 중: http://www.pythonscraping.com/exercises/exercise1.html
📊 응답 상태 코드: 200
📏 응답 크기: 564 문자
🔤 인코딩: ISO-8859-1
✅ 성공적으로 페이지를 가져왔습니다!

📄 HTML 미리보기:
<html>
<head>
<title>A Useful Page</title>
</head>
<body>
<h1>An Interesting Title</h1>
<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliqui...

🧪 테스트 2: https://httpbin.org/status/404
🔄 요청 중: https://httpbin.org/status/404
📊 응답 상태 코드: 404
📏 응답 크기: 0 문자
🔤 인코딩: utf-8
❌ 페이지를 찾을 수 없습니다 (404)
❌ 실패: 페이지 없음

🧪 테스트 3: https://httpbin.org/delay/2
🔄 요청 중: https://httpbin.org/delay/2
📊 응답 상태 코드: 200
📏 응답 크기: 396 문자
🔤 인코딩: utf-8
✅ 성공적으로 페이지를 가져왔습니다!

📄 HTML 미리보기:
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
 

## 4. 📝 HTML 구조 이해하기

### HTML의 기본 구조
HTML(HyperText Markup Language)은 웹페이지의 구조를 정의하는 마크업 언어입니다.

```html
<!DOCTYPE html>
<html>
<head>
    <title>페이지 제목</title>
    <meta charset="UTF-8">
</head>
<body>
    <h1 id="main-title" class="header">메인 제목</h1>
    <div class="content">
        <p>본문 내용</p>
        <ul>
            <li>목록 항목 1</li>
            <li>목록 항목 2</li>
        </ul>
    </div>
</body>
</html>
```

### HTML 요소의 구성
- **태그(Tag)**: `<태그명>내용</태그명>`
- **속성(Attribute)**: `<태그 속성="값">`
- **ID**: 고유 식별자 `id="unique-name"`
- **Class**: 스타일링/그룹핑용 `class="group-name"`

### 크롤링 관점에서 중요한 HTML 요소들

| 태그 | 용도 | 예시 |
|------|------|------|
| `<title>` | 페이지 제목 | `<title>뉴스 제목</title>` |
| `<h1>~<h6>` | 제목/헤더 | `<h1>메인 제목</h1>` |
| `<p>` | 문단 | `<p>본문 내용</p>` |
| `<div>` | 구역 나누기 | `<div class="article">...</div>` |
| `<span>` | 인라인 요소 | `<span class="price">1,000원</span>` |
| `<table>` | 표 | `<table><tr><td>데이터</td></tr></table>` |
| `<ul>`, `<li>` | 목록 | `<ul><li>항목</li></ul>` |
| `<a>` | 링크 | `<a href="url">링크텍스트</a>` |

### CSS 선택자 기초
크롤링에서 원하는 데이터를 정확히 찾기 위해 CSS 선택자를 이해해야 합니다:

- **태그 선택자**: `h1`, `p`, `div`
- **ID 선택자**: `#main-title`
- **클래스 선택자**: `.content`, `.header`
- **속성 선택자**: `[href]`, `[class="content"]`
- **조합 선택자**: `div.content p` (div 안의 p 태그)


## 5. 🍲 BeautifulSoup 기초 파싱

### BeautifulSoup란?
BeautifulSoup는 HTML과 XML 문서를 파싱하여 구조화된 데이터로 변환해주는 Python 라이브러리입니다.

### 주요 특징
- **간단한 API**: 직관적이고 사용하기 쉬운 인터페이스
- **관대한 파싱**: 잘못된 HTML도 잘 처리
- **다양한 파서**: html.parser, lxml, html5lib 지원
- **CSS 선택자**: CSS 선택자를 사용한 요소 선택

### 기본 사용법
```python
from bs4 import BeautifulSoup

# HTML 파싱
soup = BeautifulSoup(html_content, 'html.parser')

# 요소 찾기
soup.find('태그명')              # 첫 번째 요소
soup.find_all('태그명')          # 모든 요소
soup.select('CSS선택자')         # CSS 선택자로 찾기
```

### find() vs find_all() vs select()
- **find()**: 조건에 맞는 **첫 번째 요소만** 반환
- **find_all()**: 조건에 맞는 **모든 요소를 리스트로** 반환
- **select()**: **CSS 선택자**를 사용하여 요소 찾기


In [3]:
# 🥄 BeautifulSoup 기초 실습

from bs4 import BeautifulSoup
import requests

# 실습용 샘플 HTML 생성
sample_html = """
<!DOCTYPE html>
<html>
<head>
    <title>온라인 서점 - 베스트셀러</title>
    <meta charset="UTF-8">
</head>
<body>
    <header>
        <h1 id="main-title" class="site-title">📚 온라인 서점</h1>
        <nav class="navigation">
            <a href="/books">도서</a>
            <a href="/authors">작가</a>
        </nav>
    </header>
    
    <main class="content">
        <section class="bestsellers">
            <h2 class="section-title">이번 주 베스트셀러</h2>
            
            <div class="book" data-id="1">
                <h3 class="book-title">파이썬 완벽 가이드</h3>
                <p class="author">김개발</p>
                <span class="price">25,000원</span>
                <div class="rating">⭐⭐⭐⭐⭐ (4.8)</div>
            </div>
            
            <div class="book" data-id="2">
                <h3 class="book-title">웹 크롤링 마스터</h3>
                <p class="author">이크롤링</p>
                <span class="price">30,000원</span>
                <div class="rating">⭐⭐⭐⭐ (4.5)</div>
            </div>
            
            <div class="book" data-id="3">
                <h3 class="book-title">데이터 분석 실전</h3>
                <p class="author">박데이터</p>
                <span class="price">28,000원</span>
                <div class="rating">⭐⭐⭐⭐⭐ (4.9)</div>
            </div>
        </section>
        
        <aside class="sidebar">
            <h3>추천 카테고리</h3>
            <ul class="categories">
                <li><a href="/category/programming">프로그래밍</a></li>
                <li><a href="/category/data-science">데이터 사이언스</a></li>
                <li><a href="/category/web">웹 개발</a></li>
            </ul>
        </aside>
    </main>
    
    <footer>
        <p>&copy; 2024 온라인 서점. All rights reserved.</p>
    </footer>
</body>
</html>
"""

# BeautifulSoup로 파싱
print("🔄 HTML 파싱 중...")
soup = BeautifulSoup(sample_html, 'html.parser')
print("✅ 파싱 완료!")

print("\n" + "="*60)
print("📖 BeautifulSoup 기본 메소드 실습")
print("="*60)

# 1. find() - 첫 번째 요소만 찾기
print("\n1️⃣ find() - 첫 번째 요소 찾기")
print("-" * 30)

title = soup.find('title')
if title:
    print(f"페이지 제목: {title.text}")

first_book_title = soup.find('h3', class_='book-title')
if first_book_title:
    print(f"첫 번째 책 제목: {first_book_title.text}")

# 2. find_all() - 모든 요소 찾기
print("\n2️⃣ find_all() - 모든 요소 찾기")
print("-" * 30)

all_book_titles = soup.find_all('h3', class_='book-title')
print(f"총 {len(all_book_titles)}권의 책이 있습니다:")
for i, title in enumerate(all_book_titles, 1):
    print(f"  {i}. {title.text}")

# 3. 속성으로 찾기
print("\n3️⃣ 속성으로 요소 찾기")
print("-" * 30)

# ID로 찾기
main_title = soup.find('h1', id='main-title')
if main_title:
    print(f"메인 제목: {main_title.text}")

# 여러 속성으로 찾기
books_with_data_id = soup.find_all('div', {'class': 'book', 'data-id': True})
print(f"data-id 속성이 있는 책: {len(books_with_data_id)}권")

# 4. 텍스트 내용과 속성 값 가져오기
print("\n4️⃣ 텍스트와 속성 값 추출")
print("-" * 30)

for book in soup.find_all('div', class_='book'):
    title = book.find('h3', class_='book-title').text
    author = book.find('p', class_='author').text
    price = book.find('span', class_='price').text
    rating = book.find('div', class_='rating').text
    book_id = book.get('data-id')  # 속성 값 가져오기
    
    print(f"📗 도서 ID: {book_id}")
    print(f"   제목: {title}")
    print(f"   저자: {author}")
    print(f"   가격: {price}")
    print(f"   평점: {rating}")
    print()

print("🎯 BeautifulSoup 기초 실습 완료!")


🔄 HTML 파싱 중...
✅ 파싱 완료!

📖 BeautifulSoup 기본 메소드 실습

1️⃣ find() - 첫 번째 요소 찾기
------------------------------
페이지 제목: 온라인 서점 - 베스트셀러
첫 번째 책 제목: 파이썬 완벽 가이드

2️⃣ find_all() - 모든 요소 찾기
------------------------------
총 3권의 책이 있습니다:
  1. 파이썬 완벽 가이드
  2. 웹 크롤링 마스터
  3. 데이터 분석 실전

3️⃣ 속성으로 요소 찾기
------------------------------
메인 제목: 📚 온라인 서점
data-id 속성이 있는 책: 3권

4️⃣ 텍스트와 속성 값 추출
------------------------------
📗 도서 ID: 1
   제목: 파이썬 완벽 가이드
   저자: 김개발
   가격: 25,000원
   평점: ⭐⭐⭐⭐⭐ (4.8)

📗 도서 ID: 2
   제목: 웹 크롤링 마스터
   저자: 이크롤링
   가격: 30,000원
   평점: ⭐⭐⭐⭐ (4.5)

📗 도서 ID: 3
   제목: 데이터 분석 실전
   저자: 박데이터
   가격: 28,000원
   평점: ⭐⭐⭐⭐⭐ (4.9)

🎯 BeautifulSoup 기초 실습 완료!


## 6. 🎯 CSS 선택자와 고급 파싱 기법

### CSS 선택자의 강력함
CSS 선택자를 사용하면 복잡한 HTML 구조에서도 정확한 요소를 찾을 수 있습니다.

### BeautifulSoup의 select() 메소드
`soup.select(CSS선택자)`를 사용하여 더 정교한 요소 선택이 가능합니다.

### 주요 CSS 선택자 패턴

| 선택자 | 의미 | 예시 |
|--------|------|------|
| `tag` | 태그 선택 | `soup.select('h1')` |
| `.class` | 클래스 선택 | `soup.select('.book-title')` |
| `#id` | ID 선택 | `soup.select('#main-title')` |
| `[attribute]` | 속성 선택 | `soup.select('[data-id]')` |
| `parent > child` | 직접 자식 | `soup.select('div > h3')` |
| `ancestor descendant` | 후손 | `soup.select('section h3')` |
| `:nth-child(n)` | n번째 자식 | `soup.select('li:nth-child(2)')` |

### 고급 데이터 추출 기법
- **텍스트 정제**: `strip()`, `replace()` 활용
- **정규표현식**: 복잡한 패턴 추출
- **상대 위치**: 형제/부모 요소 탐색


In [4]:
# 🎯 CSS 선택자 마스터 클래스

import re
from bs4 import BeautifulSoup

# 앞에서 사용한 온라인 서점 HTML을 재사용
# (soup 변수가 이미 정의되어 있다고 가정)

print("🔍 CSS 선택자 실습 시작!")
print("=" * 60)

# 1. 기본 선택자 비교
print("\n1️⃣ 기본 선택자 비교 (find vs select)")
print("-" * 40)

# find() 방식
find_title = soup.find('h3', class_='book-title')
print(f"find() 결과: {find_title.text if find_title else 'None'}")

# select() 방식 - CSS 선택자
select_title = soup.select('.book-title')[0]  # 첫 번째 요소
print(f"select() 결과: {select_title.text}")

# 2. 다양한 CSS 선택자 실습
print("\n2️⃣ 다양한 CSS 선택자")
print("-" * 40)

# 클래스 선택자
book_titles = soup.select('.book-title')
print(f"📚 모든 책 제목 ({len(book_titles)}개):")
for i, title in enumerate(book_titles, 1):
    print(f"  {i}. {title.text}")

# ID 선택자
main_title = soup.select('#main-title')
if main_title:
    print(f"\n🏠 메인 제목: {main_title[0].text}")

# 속성 선택자
books_with_data_id = soup.select('[data-id]')
print(f"\n📊 data-id 속성이 있는 요소: {len(books_with_data_id)}개")

# 3. 조합 선택자 (Combinator)
print("\n3️⃣ 조합 선택자")
print("-" * 40)

# 후손 선택자 (공백)
section_titles = soup.select('section h3')
print(f"📖 section 안의 h3 태그: {len(section_titles)}개")

# 직접 자식 선택자 (>)
direct_children = soup.select('.book > h3')
print(f"📘 .book의 직접 자식 h3: {len(direct_children)}개")

# 인접 형제 선택자 (+) - 바로 다음 형제
next_siblings = soup.select('h3 + p')  # h3 바로 다음의 p 태그
print(f"👥 h3 바로 다음의 p 태그: {len(next_siblings)}개")

# 4. 가상 선택자 (Pseudo-selector)
print("\n4️⃣ 가상 선택자")
print("-" * 40)

# n번째 자식
first_book = soup.select('.book:first-child')
if first_book:
    title = first_book[0].select('.book-title')[0].text
    print(f"🥇 첫 번째 책: {title}")

last_book = soup.select('.book:last-child')
if last_book:
    title = last_book[0].select('.book-title')[0].text
    print(f"🥉 마지막 책: {title}")

# n번째 특정 자식
second_book = soup.select('.book:nth-child(2)')
if second_book:
    title = second_book[0].select('.book-title')[0].text
    print(f"🥈 두 번째 책: {title}")

# 5. 복합 조건 선택
print("\n5️⃣ 복합 조건 선택")
print("-" * 40)

# 여러 클래스 조건
# 만약 'book premium' 클래스가 있다면
# premium_books = soup.select('.book.premium')

# 속성 값으로 선택
book_1 = soup.select('[data-id="1"]')
if book_1:
    title = book_1[0].select('.book-title')[0].text
    print(f"📗 ID가 1인 책: {title}")

# 6. 텍스트 기반 고급 추출
print("\n6️⃣ 고급 데이터 추출")
print("-" * 40)

def extract_book_info(book_element):
    """책 정보를 구조화하여 추출하는 함수"""
    info = {}
    
    # 기본 정보 추출
    info['id'] = book_element.get('data-id')
    info['title'] = book_element.select('.book-title')[0].text.strip()
    info['author'] = book_element.select('.author')[0].text.strip()
    
    # 가격에서 숫자만 추출 (정규표현식 사용)
    price_text = book_element.select('.price')[0].text
    price_match = re.search(r'[\d,]+', price_text)
    info['price'] = int(price_match.group().replace(',', '')) if price_match else 0
    
    # 평점에서 숫자 추출
    rating_text = book_element.select('.rating')[0].text
    rating_match = re.search(r'\((\d+\.\d+)\)', rating_text)
    info['rating'] = float(rating_match.group(1)) if rating_match else 0.0
    
    # 별 개수 계산
    star_count = rating_text.count('⭐')
    info['stars'] = star_count
    
    return info

# 모든 책 정보 추출
books_data = []
for book in soup.select('.book'):
    book_info = extract_book_info(book)
    books_data.append(book_info)

# 추출된 데이터 출력
print("📚 추출된 책 정보:")
for book in books_data:
    print(f"  📖 {book['title']}")
    print(f"     저자: {book['author']}")
    print(f"     가격: {book['price']:,}원")
    print(f"     평점: {book['rating']} ({book['stars']}⭐)")
    print()

# 7. 데이터 분석
print("7️⃣ 간단한 데이터 분석")
print("-" * 40)

total_books = len(books_data)
avg_price = sum(book['price'] for book in books_data) / total_books
avg_rating = sum(book['rating'] for book in books_data) / total_books
highest_rated = max(books_data, key=lambda x: x['rating'])

print(f"📊 총 도서 수: {total_books}권")
print(f"💰 평균 가격: {avg_price:,.0f}원")
print(f"⭐ 평균 평점: {avg_rating:.1f}")
print(f"🏆 최고 평점 도서: {highest_rated['title']} ({highest_rated['rating']}점)")

print("\n🎯 CSS 선택자 마스터 클래스 완료!")


🔍 CSS 선택자 실습 시작!

1️⃣ 기본 선택자 비교 (find vs select)
----------------------------------------
find() 결과: 파이썬 완벽 가이드
select() 결과: 파이썬 완벽 가이드

2️⃣ 다양한 CSS 선택자
----------------------------------------
📚 모든 책 제목 (3개):
  1. 파이썬 완벽 가이드
  2. 웹 크롤링 마스터
  3. 데이터 분석 실전

🏠 메인 제목: 📚 온라인 서점

📊 data-id 속성이 있는 요소: 3개

3️⃣ 조합 선택자
----------------------------------------
📖 section 안의 h3 태그: 3개
📘 .book의 직접 자식 h3: 3개
👥 h3 바로 다음의 p 태그: 3개

4️⃣ 가상 선택자
----------------------------------------
🥉 마지막 책: 데이터 분석 실전
🥈 두 번째 책: 파이썬 완벽 가이드

5️⃣ 복합 조건 선택
----------------------------------------
📗 ID가 1인 책: 파이썬 완벽 가이드

6️⃣ 고급 데이터 추출
----------------------------------------
📚 추출된 책 정보:
  📖 파이썬 완벽 가이드
     저자: 김개발
     가격: 25,000원
     평점: 4.8 (5⭐)

  📖 웹 크롤링 마스터
     저자: 이크롤링
     가격: 30,000원
     평점: 4.5 (4⭐)

  📖 데이터 분석 실전
     저자: 박데이터
     가격: 28,000원
     평점: 4.9 (5⭐)

7️⃣ 간단한 데이터 분석
----------------------------------------
📊 총 도서 수: 3권
💰 평균 가격: 27,667원
⭐ 평균 평점: 4.7
🏆 최고 평점 도서: 데이터 분석 실전 (4.9점)

🎯 CSS 선택자 마스터 클래스 완료!

### 실제 웹사이트 크롤링 전략

이제 실제 웹사이트에서 데이터를 추출해보겠습니다. 안전하고 윤리적인 크롤링을 위한 원칙들을 적용합니다.

### 크롤링 전략 수립
1. **robots.txt 확인**: `사이트주소/robots.txt`
2. **요청 간격 조절**: 서버 부하 방지
3. **User-Agent 설정**: 적절한 브라우저 헤더
4. **에러 처리**: 네트워크 오류, 파싱 오류 대응
5. **데이터 검증**: 추출한 데이터의 유효성 확인

### 실습 대상
- **HTTPBin**: 테스트용 HTTP 서비스
- **Python Scraping 예제 사이트**: 학습용 사이트
- **공개 API**: JSON 데이터 처리 연습


In [5]:
# 🚀 실전 크롤링 프로젝트

import requests
from bs4 import BeautifulSoup
import time
import json
from urllib.parse import urljoin, urlparse

class WebCrawler:
    """안전하고 효율적인 웹 크롤러 클래스"""
    
    def __init__(self, delay=1):
        self.session = requests.Session()
        self.delay = delay  # 요청 간격 (초)
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
    
    def fetch_page(self, url, timeout=10):
        """안전하게 웹페이지를 가져오는 메소드"""
        try:
            print(f"🔄 요청: {url}")
            response = self.session.get(url, timeout=timeout)
            
            if response.status_code == 200:
                print(f"✅ 성공 (크기: {len(response.text):,} 문자)")
                time.sleep(self.delay)  # 요청 간격 조절
                return response
            else:
                print(f"❌ HTTP {response.status_code} 오류")
                return None
                
        except requests.exceptions.RequestException as e:
            print(f"💥 요청 오류: {e}")
            return None
    
    def parse_html(self, html_content):
        """HTML을 파싱하여 BeautifulSoup 객체 반환"""
        return BeautifulSoup(html_content, 'html.parser')

# 크롤러 인스턴스 생성
crawler = WebCrawler(delay=2)  # 2초 간격

print("🎯 실전 크롤링 프로젝트 시작!")
print("=" * 60)

# 1. 간단한 HTML 페이지 크롤링
print("\n1️⃣ 기본 HTML 페이지 크롤링")
print("-" * 40)

response = crawler.fetch_page("http://www.pythonscraping.com/exercises/exercise1.html")
if response:
    soup = crawler.parse_html(response.text)
    title = soup.find('title')
    h1 = soup.find('h1')
    
    print(f"📄 페이지 제목: {title.text if title else '없음'}")
    print(f"🔤 메인 헤딩: {h1.text if h1 else '없음'}")

# 2. JSON API 데이터 크롤링
print("\n2️⃣ JSON API 데이터 처리")
print("-" * 40)

response = crawler.fetch_page("https://httpbin.org/json")
if response:
    try:
        data = response.json()  # JSON 파싱
        print("📊 JSON 데이터:")
        print(json.dumps(data, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("❌ JSON 파싱 실패")

# 3. 복잡한 구조의 페이지 크롤링
print("\n3️⃣ 복잡한 HTML 구조 분석")
print("-" * 40)

response = crawler.fetch_page("http://www.pythonscraping.com/pages/warandpeace.html")
if response:
    soup = crawler.parse_html(response.text)
    
    # 여러 데이터 추출
    name_list = soup.find_all("span", {"class": "green"})
    print(f"🟢 녹색 텍스트 ({len(name_list)}개):")
    for i, name in enumerate(name_list[:5], 1):  # 처음 5개만
        print(f"  {i}. {name.text.strip()}")
    
    if len(name_list) > 5:
        print(f"  ... 및 {len(name_list) - 5}개 더")

# 4. 링크 수집하기
print("\n4️⃣ 페이지 내 링크 수집")
print("-" * 40)

response = crawler.fetch_page("http://www.pythonscraping.com/pages/page3.html")
if response:
    soup = crawler.parse_html(response.text)
    
    # 모든 링크 수집
    links = soup.find_all('a', href=True)
    internal_links = []
    external_links = []
    
    base_url = "http://www.pythonscraping.com"
    
    for link in links:
        href = link['href']
        full_url = urljoin(base_url, href)
        link_text = link.text.strip()
        
        if urlparse(full_url).netloc == urlparse(base_url).netloc:
            internal_links.append((link_text, full_url))
        else:
            external_links.append((link_text, full_url))
    
    print(f"🔗 내부 링크 ({len(internal_links)}개):")
    for text, url in internal_links[:3]:
        print(f"  📄 {text}: {url}")
    
    print(f"\n🌐 외부 링크 ({len(external_links)}개):")
    for text, url in external_links[:3]:
        print(f"  🔗 {text}: {url}")

# 5. 표(Table) 데이터 추출
print("\n5️⃣ 테이블 데이터 구조화")
print("-" * 40)

# 테이블이 있는 페이지에서 데이터 추출
if response:  # 이전 response 재사용
    soup = crawler.parse_html(response.text)
    
    # 테이블 찾기
    try:
        table = soup.find('table')
        if table:
            rows = table.find_all('tr')  # type: ignore
            table_data = []
            
            for row in rows:
                cells = row.find_all(['th', 'td'])  # type: ignore
                row_data = [cell.text.strip() for cell in cells]
                if row_data:  # 빈 행이 아닌 경우만
                    table_data.append(row_data)
            
            print(f"📊 테이블 데이터 ({len(table_data)}행):")
            for i, row in enumerate(table_data[:5], 1):  # 처음 5행만
                print(f"  {i}. {' | '.join(row)}")
            
            if len(table_data) > 5:
                print(f"  ... 및 {len(table_data) - 5}행 더")
        else:
            print("📊 테이블이 없습니다.")
    except Exception as e:
        print(f"📊 테이블 처리 중 오류: {e}")

# 6. 에러 처리 테스트
print("\n6️⃣ 에러 처리 테스트")
print("-" * 40)

# 존재하지 않는 페이지 요청
error_response = crawler.fetch_page("http://www.pythonscraping.com/nonexistent-page")

# 잘못된 도메인 요청
error_response2 = crawler.fetch_page("http://this-domain-does-not-exist-12345.com")

print("\n🎯 실전 크롤링 프로젝트 완료!")
print("\n💡 핵심 포인트:")
print("  - 요청 간격 조절로 서버 부하 방지")
print("  - 다양한 에러 상황에 대한 적절한 처리")
print("  - 구조화된 데이터 추출 및 정제")
print("  - JSON과 HTML 두 가지 형태의 데이터 처리")


🎯 실전 크롤링 프로젝트 시작!

1️⃣ 기본 HTML 페이지 크롤링
----------------------------------------
🔄 요청: http://www.pythonscraping.com/exercises/exercise1.html
✅ 성공 (크기: 564 문자)
📄 페이지 제목: A Useful Page
🔤 메인 헤딩: An Interesting Title

2️⃣ JSON API 데이터 처리
----------------------------------------
🔄 요청: https://httpbin.org/json
✅ 성공 (크기: 429 문자)
📊 JSON 데이터:
{
  "slideshow": {
    "author": "Yours Truly",
    "date": "date of publication",
    "slides": [
      {
        "title": "Wake up to WonderWidgets!",
        "type": "all"
      },
      {
        "items": [
          "Why <em>WonderWidgets</em> are great",
          "Who <em>buys</em> WonderWidgets"
        ],
        "title": "Overview",
        "type": "all"
      }
    ],
    "title": "Sample Slide Show"
  }
}

3️⃣ 복잡한 HTML 구조 분석
----------------------------------------
🔄 요청: http://www.pythonscraping.com/pages/warandpeace.html
✅ 성공 (크기: 11,723 문자)
🟢 녹색 텍스트 (41개):
  1. Anna
Pavlovna Scherer
  2. Empress Marya
Fedorovna
  3. Prince Vasili Kuragin
  4

## 8. ⚠️ 에러 처리와 예외 상황 대응

### 웹 크롤링에서 발생할 수 있는 문제들

웹 크롤링은 다양한 예상치 못한 상황들이 발생할 수 있는 작업입니다. 안정적인 크롤러를 만들기 위해서는 적절한 에러 처리가 필수입니다.

#### 주요 예외 상황들:
- **네트워크 오류**: 연결 실패, 타임아웃
- **HTTP 오류**: 404, 403, 500 등
- **파싱 오류**: 예상과 다른 HTML 구조
- **인코딩 문제**: 문자 깨짐
- **동적 콘텐츠**: JavaScript로 생성되는 내용
- **봇 차단**: CAPTCHA, IP 차단

#### 방어적 프로그래밍 원칙:
1. **항상 예외 처리**: try-except 블록 사용
2. **데이터 검증**: 예상한 형태의 데이터인지 확인
3. **안전한 접근**: None 체크, 배열 인덱스 검증
4. **재시도 로직**: 일시적 오류에 대한 재시도
5. **로깅**: 문제 상황 기록 및 디버깅


## 9. ⚖️ 크롤링 윤리와 법적 주의사항

### 🚨 반드시 지켜야 할 윤리적 원칙

웹 크롤링은 강력한 도구이지만, 책임감 있게 사용해야 합니다.

#### 법적 준수사항:
- **저작권 보호**: 콘텐츠의 저작권 존중
- **개인정보 보호**: GDPR, 개인정보보호법 준수
- **이용약관 확인**: 웹사이트의 Terms of Service 검토
- **robots.txt 준수**: 사이트의 크롤링 정책 확인

#### 기술적 윤리:
- **서버 부하 최소화**: 적절한 요청 간격 유지
- **대역폭 고려**: 불필요한 대용량 파일 다운로드 금지
- **User-Agent 명시**: 본인의 정체를 숨기지 않기
- **재시도 제한**: 무한 재시도로 서버 공격하지 않기

#### robots.txt 확인 방법
```python
# robots.txt 확인 예제
import requests

def check_robots_txt(domain):
    robots_url = f"{domain}/robots.txt"
    try:
        response = requests.get(robots_url)
        if response.status_code == 200:
            print(f"🤖 {domain}의 robots.txt:")
            print(response.text)
        else:
            print(f"❌ robots.txt를 찾을 수 없습니다.")
    except Exception as e:
        print(f"💥 오류: {e}")

# 예시: check_robots_txt("http://www.example.com")
```

### 💡 책임감 있는 크롤링을 위한 체크리스트

✅ **법적 검토**
- [ ] 대상 사이트의 이용약관 확인
- [ ] robots.txt 정책 준수
- [ ] 개인정보 포함 여부 확인
- [ ] 저작권 침해 가능성 검토

✅ **기술적 고려**
- [ ] 적절한 요청 간격 설정 (최소 1초)
- [ ] User-Agent 헤더 명시
- [ ] 에러 처리 및 재시도 로직 구현
- [ ] 서버 응답 코드 확인

✅ **데이터 사용**
- [ ] 수집 목적에 맞는 최소한의 데이터만 추출
- [ ] 개인정보 처리 시 적절한 보안 조치
- [ ] 데이터 보관 기간 및 파기 정책 수립


## 10. 🎓 종합 정리 및 실전 프로젝트 제안

### 🏆 학습 완료! 축하합니다!

이 노트북을 통해 웹 크롤링의 전체 과정을 체계적으로 학습했습니다.

### 📚 학습한 핵심 내용

#### 1. 기초 개념
- ✅ HTTP 통신과 웹 크롤링의 원리
- ✅ requests 모듈을 이용한 웹페이지 요청
- ✅ 상태 코드와 에러 처리

#### 2. HTML 파싱
- ✅ BeautifulSoup 라이브러리 사용법
- ✅ find(), find_all(), select() 메소드
- ✅ CSS 선택자를 이용한 정교한 요소 선택

#### 3. 고급 기법
- ✅ 정규표현식을 이용한 데이터 정제
- ✅ 구조화된 데이터 추출 및 분석
- ✅ 세션 관리와 헤더 설정

#### 4. 실전 응용
- ✅ 안전한 크롤링 클래스 구현
- ✅ JSON과 HTML 데이터 처리
- ✅ 링크 수집 및 페이지 탐색

#### 5. 윤리와 법적 고려
- ✅ robots.txt 확인 방법
- ✅ 책임감 있는 크롤링 원칙
- ✅ 개인정보 보호와 저작권 준수

---

### 🚀 다음 단계: 실전 프로젝트 제안

이제 배운 지식을 활용해 실제 프로젝트를 시도해보세요!

#### 🌟 초급 프로젝트
1. **날씨 정보 수집기**
   - 기상청 또는 날씨 사이트에서 오늘의 날씨 정보 추출
   - 온도, 습도, 강수량 등을 정리해서 텍스트 파일로 저장

2. **뉴스 제목 모음집**
   - 뉴스 사이트에서 오늘의 주요 뉴스 제목들 수집
   - 카테고리별로 분류하여 정리

3. **온라인 쇼핑몰 가격 비교**
   - 특정 상품의 가격을 여러 쇼핑몰에서 수집
   - 최저가와 평균가 계산

#### 🔥 중급 프로젝트
4. **부동산 시세 분석기**
   - 부동산 정보 사이트에서 지역별 매매가 정보 수집
   - 시계열 데이터로 변환하여 트렌드 분석

5. **채용 정보 모니터링**
   - 채용 사이트에서 특정 직무의 공고 정보 수집
   - 요구 스킬, 연봉 범위 등을 분석하여 인사이트 도출

6. **소셜 미디어 트렌드 분석**
   - 공개 API를 활용한 트렌드 키워드 수집
   - 시간대별, 지역별 인기 토픽 분석

#### 🚀 고급 프로젝트
7. **학술 논문 메타데이터 수집**
   - arXiv, Google Scholar 등에서 특정 분야 논문 정보 수집
   - 저자 네트워크, 인용 관계 분석

8. **전자상거래 리뷰 분석**
   - 제품 리뷰 데이터 대량 수집
   - 감정 분석과 키워드 추출을 통한 제품 평가

---

### 🛠️ 추가 학습 로드맵

#### 다음에 배울 고급 기술들:

1. **Selenium WebDriver**
   - JavaScript 렌더링이 필요한 동적 웹페이지 크롤링
   - 브라우저 자동화를 통한 복잡한 상호작용

2. **Scrapy Framework**
   - 대규모 크롤링을 위한 전문 프레임워크
   - 분산 크롤링과 데이터 파이프라인

3. **데이터 저장 및 처리**
   - 데이터베이스 연동 (MySQL, MongoDB)
   - 데이터 분석 라이브러리 (pandas, numpy)

4. **성능 최적화**
   - 비동기 프로그래밍 (asyncio, aiohttp)
   - 멀티스레딩과 멀티프로세싱

5. **클라우드 배포**
   - AWS, GCP를 이용한 크롤러 배포
   - 스케줄링과 모니터링

---

### 💪 마무리 메시지

웹 크롤링은 데이터 과학, 비즈니스 인텔리전스, 마케팅 등 다양한 분야에서 활용되는 실용적인 기술입니다. 

**기억하세요:**
- 항상 윤리적이고 법적인 기준을 준수하세요
- 작은 프로젝트부터 시작해서 점진적으로 복잡한 것에 도전하세요
- 실패를 두려워하지 말고, 문제 해결 과정에서 배우세요
- 커뮤니티와 함께 지식을 공유하고 성장하세요

**🎯 여러분의 첫 번째 크롤링 프로젝트를 시작해보세요!**

---

*"데이터는 21세기의 석유다. 하지만 석유와 달리 데이터는 사용할수록 더 가치있어진다."*

**Happy Crawling! 🕷️✨**
